<a href="https://colab.research.google.com/github/avalenciacu/SYS-2025-2/blob/main/ejercicioTransformadaZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Descargue 10 segundos de la canción de su preferencia desde youtube, y generé un filtro pasabanda (el usuario debe poder definir las frecuencias de corte) para cada uno de los filtros descritos (el usuario también debe poder  fijar los parámetros de diseño de cada filtro). Compare los resultados de los filtros estudiados en este cuaderno tipo IIR para diseño Butterworth, Chebyshev 1, Chebyshev 2, Bessel y Elíptico.

In [ ]:
# @title Instalación de librerías necesarias
!python3 -m pip install --quiet --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install --quiet soundfile

import numpy as np
import soundfile as sf
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from IPython.display import Audio
from scipy.signal import butter, cheby1, cheby2, ellip, bessel
from scipy.signal import freqz, freqz_zpk, lfilter, filtfilt, tf2zpk

In [ ]:
# @title Descarga y carga del audio (10 segundos)
link = "https://www.youtube.com/watch?v=IhnOpwOMHgk"  # Canción elegida
!yt-dlp --extract-audio -o "audio" --audio-format mp3 {link}
!ffmpeg -loglevel quiet -y -i audio.mp3 output.wav

x, fs = sf.read("output.wav")
ti, tf = 15, 25  # segundos a recortar
xs = x[int(ti*fs):int(tf*fs), :]
print(f"Frecuencia de muestreo: {fs} Hz - Duración del fragmento: {xs.shape[0]/fs:.2f} s")
Audio([xs[:,1], xs[:,0]], rate=fs)


In [ ]:
# @title Funciones de visualización de filtros
def plot_freq_response(filter_name, w, h, N):
    fig, ax1 = plt.subplots()
    ax1.set_title(f'{filter_name} - Orden {N}')
    ax1.plot(w, 20 * np.log10(abs(h)), 'b')
    ax1.set_ylabel('Magnitud [dB]', color='b')
    ax2 = ax1.twinx()
    ax2.plot(w, np.unwrap(np.angle(h)), 'g')
    ax2.set_ylabel('Fase [rad]', color='g')
    ax1.set_xlabel('Frecuencia [Hz]')
    ax1.grid()
    plt.show()

def show_zp(z, p, title='Z-plane'):
    fig, ax = plt.subplots()
    ax.plot(np.real(z), np.imag(z), 'bo', label='Zeros')
    ax.plot(np.real(p), np.imag(p), 'rx', label='Poles')
    circle = Circle((0, 0), radius=1, fill=False, color='black', ls='dashed')
    ax.add_patch(circle)
    ax.axvline(0, color='0.7')
    ax.axhline(0, color='0.7')
    ax.set_title(title)
    ax.legend()
    plt.axis('equal')
    plt.grid()
    plt.show()

In [ ]:
# @title Parámetros generales del filtro (puedes cambiarlos)
fc1 = 1100  # @param {type:"slider", min:100, max:10000, step:100}
fc2 = 4000  # @param {type:"slider", min:100, max:10000, step:100}
order = 5  # @param {type:"slider", min:1, max:10, step:1}
ripple_pass = 1  # dB @param
ripple_stop = 40  # dB @param

Wn = [fc1, fc2]

In [ ]:
# @title Diseño y visualización de los filtros IIR

def design_and_apply_filter(name, filter_func, *args, **kwargs):
    if 'output' in kwargs and kwargs['output'] == 'zpk':
        z, p, k = filter_func(*args, **kwargs)
        w, h = freqz_zpk(z, p, k, fs=fs)
        b, a = np.poly(z)*k, np.poly(p)
    else:
        b, a = filter_func(*args, **kwargs)
        w, h = freqz(b, a, fs=fs)
        z, p, _ = tf2zpk(b, a)

    plot_freq_response(name, w, h, order)
    show_zp(z, p, f'Polos y Ceros - {name}')

    xf = filtfilt(b, a, xs, axis=0)
    return xf

filtered = {}

print("Butterworth")
filtered["Butterworth"] = design_and_apply_filter("Butterworth", butter, order, Wn, btype='bandpass', fs=fs, output='zpk')

print("Chebyshev I")
filtered["Chebyshev1"] = design_and_apply_filter("Chebyshev1", cheby1, order, ripple_pass, Wn, btype='bandpass', fs=fs, output='zpk')

print("Chebyshev II")
filtered["Chebyshev2"] = design_and_apply_filter("Chebyshev2", cheby2, order, ripple_stop, Wn, btype='bandpass', fs=fs)

print("Elíptico")
filtered["Elliptic"] = design_and_apply_filter("Elliptic", ellip, order, ripple_pass, ripple_stop, Wn, btype='bandpass', fs=fs)

print("Bessel")
filtered["Bessel"] = design_and_apply_filter("Bessel", bessel, order, Wn, btype='bandpass', norm='mag', fs=fs)

In [ ]:
# @title Escuchar los resultados filtrados
from IPython.display import display

for name, audio in filtered.items():
    print(f"\nFiltro: {name}")
    display(Audio([audio[:,1], audio[:,0]], rate=fs))

# PUNTO 2
Consulte en qué consiste el método de diseño de filtros FIR por ventaneo (Ver función [firwin](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.firwin.html) y [material de apoyo](https://www.programcreek.com/python/example/100540/scipy.signal.firwin)). Realice un cuadro comparativo de las ventajas y desventajas de los filtros IIR y los FIR. Nota: Recuerde que un filtro FIR utiliza solamente raíces tipo ceros, es decir que $a_0=1$, y $a_k=0$ $\forall k\in\{1,2,\dots\}$.

El método de **diseño de filtros FIR por ventaneo** consiste en los siguientes pasos:

1. Se parte de una **respuesta al impulso ideal** $h_{\text{ideal}}[n]$ de un filtro (por ejemplo, paso bajo ideal), que típicamente es de duración infinita.
2. Para hacerla realizable, se **multiplica** por una ventana $w[n]$ de duración finita:
   
   $$
   h[n] = h_{\text{ideal}}[n] \cdot w[n]
   $$

3. Esta operación **recorta** la respuesta y suaviza los bordes, lo que limita los efectos indeseados como el fenómeno de Gibbs.
4. El filtro resultante es un **filtro FIR** (Finite Impulse Response), caracterizado por:
   
   $$
   a_0 = 1,\quad a_k = 0,\quad \forall k \in \{1,2,\dots\}
   $$

Se puede diseñar fácilmente en Python usando la función `firwin` de `scipy.signal`.

---

### Cuadro comparativo: Filtros IIR vs FIR

| Característica                    | Filtros FIR                                 | Filtros IIR                                  |
|----------------------------------|---------------------------------------------|----------------------------------------------|
| **Definición**                   | Respuesta finita al impulso                 | Respuesta infinita al impulso                 |
| **Retroalimentación**            | ❌ No utilizan                               | ✅ Utilizan                                   |
| **Estabilidad**                  | Siempre estables                            | Depende de la ubicación de polos             |
| **Fase lineal**                  | ✅ Es posible lograrla fácilmente            | ❌ Difícil de lograr                          |
| **Complejidad computacional**    | Alta (mayor orden requerido)                | Baja (con bajo orden se logra buena respuesta)|
| **Precisión en bandas**          | Transiciones más amplias                    | Transiciones más abruptas y selectivas       |
| **Aplicaciones típicas**         | Audio, imagen, comunicaciones (fase crítica)| Control, radar, sistemas embebidos           |

# PUNTO 3
Incluya la implementación del filtro `firwin` en el punto 1.

In [ ]:
# @title Filtro FIR pasabanda usando firwin
from scipy.signal import firwin, freqz

# Número de coeficientes (orden + 1)
numtaps = 101  # Puedes ajustar este valor

# Diseño del filtro pasabanda
fir_coeffs = firwin(numtaps, cutoff=Wn, fs=fs, pass_zero=False, window='hamming')

# Respuesta en frecuencia
w, h = freqz(fir_coeffs, fs=fs)

# Visualización
plot_freq_response('FIR (firwin)', w, h, numtaps - 1)

# Aplicación al audio (usamos filtfilt para no introducir retardo)
xf_fir = filtfilt(fir_coeffs, [1.0], xs, axis=0)

# Guardamos el resultado para escuchar
filtered['FIR (firwin)'] = xf_fir

In [ ]:
# @title Reproducir audio filtrado (incluyendo FIR)
for name, audio in filtered.items():
    print(f"\nFiltro: {name}")
    display(Audio([audio[:,1], audio[:,0]], rate=fs))